# DOCS:

[DOCS](https://rapidapi.com/rphrp1985/api/newsnow)

In [1]:
import os

def excluir_arquivos_em_pasta(pasta):
    # Verifica se a pasta existe
    if os.path.exists(pasta):
        # Lista os arquivos na pasta
        arquivos = os.listdir(pasta)
        
        # Itera sobre os arquivos e os exclui
        for arquivo in arquivos:
            caminho_arquivo = os.path.join(pasta, arquivo)
            if os.path.isfile(caminho_arquivo):
                os.remove(caminho_arquivo)
            else:
                print(f"{caminho_arquivo} não é um arquivo e não será excluído.")
        
        print("Arquivos excluídos com sucesso.")
    else:
        print(f"A pasta {pasta} não existe.")

In [2]:
# Substitua 'caminho_para_pasta' pelo caminho da pasta que você deseja verificar e limpar.
excluir_arquivos_em_pasta('temp_news')

from datetime import datetime, timedelta
import requests 

def getNews(url: str, query: str, page: int = 1, from_date: str = None, to_date: str = None):
    # Verifica se from_date e to_date não foram fornecidos e define-os como ontem
    if from_date is None: 
        yesterday = datetime.now() - timedelta(days=1)
        from_date = yesterday.strftime('%d/%m/%Y')
    
    if to_date is None: 
        yesterday = datetime.now() - timedelta(days=1)
        to_date = yesterday.strftime('%d/%m/%Y')
    
    # Converte as datas de string (dd/MM/yyyy) para objetos datetime
    from_date = datetime.strptime(from_date, '%d/%m/%Y')
    to_date = datetime.strptime(to_date, '%d/%m/%Y')

    # Formate as datas para o formato esperado
    from_date_str = from_date.strftime('%d/%m/%Y')
    to_date_str = to_date.strftime('%d/%m/%Y')

    payload = {
        "query": query,
        "page": page,
        "time_bounded": True,
        "from_date": from_date_str,
        "to_date": to_date_str,
        "location": "",
        "category": "",
        "source": ""
    }
    headers = {
        "content-type": "application/json",
        "X-RapidAPI-Key": os.getenv("RAPIDAPI"),
        "X-RapidAPI-Host": "newsnow.p.rapidapi.com"
    }

    response = requests.post(url, json=payload, headers=headers)
    response = response.json()['news']
    return response

Arquivos excluídos com sucesso.


In [3]:
import json
from dotenv import load_dotenv

load_dotenv()

url = "https://newsnow.p.rapidapi.com/newsv2"

#Physiology of the digestive tract in dogs
query = input("Qual o termo que você quer pesquisar?")

bGetNews = True

i = 1
while bGetNews:
    _ = getNews(url = url,query = query, page = i, from_date='01/01/2023', to_date = '09/10/2023')
    
    if len(_):
        jsonData = json.dumps(_)
        with open(f'temp_news/newsnow_{str(i)}.json', 'w') as f:
            f.write(jsonData)
    else:
        bGetNews = False
    i += 1

In [4]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import Window as W

from pyspark.sql.types import StructType, StructField, StringType, IntegerType
# Configurando a sessão Spark
spark = SparkSession.builder.appName("Nubank_Analytics_Eng").getOrCreate()

23/10/09 23:52:54 WARN Utils: Your hostname, billy-virus resolves to a loopback address: 127.0.1.1; using 192.168.1.27 instead (on interface wlp0s20f3)
23/10/09 23:52:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/09 23:52:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
newsnow = spark.read.option("multiline","true").json("temp_news/*")
newsnow = (
    newsnow
)
newsnow.cache()
newsnow.repartition(1)

DataFrame[body: string, date: string, image: string, source: string, title: string, url: string]

In [6]:
from translate import Translator
from googletrans import Translator as Tgoogle
import re

# Função para remover caracteres não alfanuméricos
def remove_non_alphanumeric(text):
    return re.sub(r'[^a-zA-Z0-9\s]', '', text)

# Registrar a função UDF
remove_non_alphanumeric_udf = F.udf(remove_non_alphanumeric, StringType())

# Função para traduzir uma coluna
def translate_column_translate(text, target_lang='pt-br'):
    translator = Translator(key=os.getenv('MYMEMORYKEY'), to_lang=target_lang)
    return translator.translate(text)

# Função para traduzir uma coluna
def translate_column_google(text, source_lang='en', target_lang='pt_br'):
    try:
        if text is not None:
            translator = Tgoogle()
            translated = translator.translate(text, src=source_lang, dest=target_lang)
            return translated.text
    except Exception as e:
        print(f"Erro na tradução: {e}")
    return text

# Registrar a função UDF
translate_udf_google = F.udf(translate_column_google, StringType())

# Registrar a função UDF
translate_udf_translate = F.udf(translate_column_translate, StringType())

# Use a função para traduzir as colunas desejadas
newsnow_translate = (
    newsnow
    .withColumn("body", remove_non_alphanumeric_udf(F.col("body")))
    .withColumn("title", remove_non_alphanumeric_udf(F.col("title")))
    .withColumn("title-pt_br-translate", F.lit(translate_udf_translate(F.col("title"))))
    .withColumn("title-pt_br-google", F.lit(translate_udf_google(F.col("title"))))
    .withColumn('title-pt_br', F.when(F.col('title')==F.col('title-pt_br-google'), F.col('title-pt_br-translate')).otherwise(F.col('title-pt_br-google')))
    .withColumn("body-pt_br-translate", F.lit(translate_udf_translate(F.col("body"))))
    .withColumn("body-pt_br-google", F.lit(translate_udf_google(F.col("body"))))
    .withColumn('body-pt_br', F.when(F.col('body')==F.col('body-pt_br-google'), F.col('body-pt_br-translate')).otherwise(F.col('body-pt_br-google')))
    .select(
        F.col('source')
        ,F.col('title')
        ,F.col('body')
        ,F.col('date')
        ,F.col('title-pt_br')
        ,F.col('body-pt_br')
        ,F.col('url')
    )
)

# Exiba o DataFrame resultante
newsnow_translate.cache()

DataFrame[source: string, title: string, body: string, date: string, title-pt_br: string, body-pt_br: string, url: string]

In [10]:
newsnow_translate.write.option('encoding', 'utf-8').partitionBy('source').mode('overwrite').parquet('./parquet_news')

In [11]:
teste = spark.read.parquet('parquet_news')
teste.show()

+--------------------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+
|               title|                body|        date|         title-pt_br|          body-pt_br|                 url|              source|
+--------------------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+
|The effect of pup...| gut barrier host...| Feb 9, 2023|O efeito da dieta...|Imunidade do hosp...|https://www.natur...|              Nature|
|Comparative trans...|The porcine gut i...|Jan 24, 2023|A transcriptômica...|O intestino porci...|https://www.natur...|              Nature|
|Detecting macroev...|Specialized diges...| Jan 5, 2023|Detectar o genoti...|Fisiologia digest...|https://www.natur...|              Nature|
|Intratumoral micr...|A study of canine...|Jan 16, 2023|Papéis intratumor...|Um estudo de tumo...|https://www.natur...|              Nature|
|Musculoskele

In [12]:
teste.filter(F.col('source')=='Nature').show(truncate=False)

+---------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------+------+
|title                                                          |body                                                                                                                                                        |date        |title-pt_br                                                                   |body-pt_br                                                                                                                       

In [13]:
spark.stop()